In [1]:
import numpy as np
import pandas as pd
import random
from subprocess import Popen, PIPE, STDOUT
import gymnasium as gym
from gymnasium import spaces
import time
from gymnasium.wrappers import TimeLimit
import gymnasium as gym
import numpy as np

import matplotlib.pyplot as plt
import os
import pickle as pkl
import random
import sys
import time
from pprint import pprint

from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import StopTrainingOnNoModelImprovement
from stable_baselines3.common.monitor import Monitor



from stable_baselines3 import DQN
from stable_baselines3.common.evaluation import evaluate_policy

In [6]:


class ShipEnv3(gym.Env):
	

	def __init__(self,scenario = None, render_mode=None ):
		
		#dictionaries helping change the values from string to number and vice versa
		self.number_to_character = {'0' : 'Anna', '1':'Babish' , '2':'Charlotte'}
		self.character_to_number = {v: k for k, v in self.number_to_character.items()}
													
		self.number_to_place = {'0' : 'ship', '1':'pirateBase' , '2':'treasurePlace'}
		self.place_to_number = {v: k for k, v in self.number_to_place.items()}

		
		self.mind_level=0
		self.scenario = scenario
		
		self.score = 0
		self.characters = ['Anna', 'Babish', 'Charlotte']
		self.places= ['ship','pirateBase','treasurePlace']

		self.chars_know = ['world_state', 'pirates_relation', 'pirates_power', 'pirates_treasure', 'pirates_knows_location_treasure', 'Anna_power', 'Anna_injured', 'Anna_state', 'Anna_at', 'Anna_treasure', 'Anna_altruism', 'Anna_ambition', 'Anna_knows_location_treasure', 'Anna_has_access_pirates', 'Babish_power', 'Babish_injured', 'Babish_state', 'Babish_at', 'Babish_treasure', 'Babish_altruism', 'Babish_ambition', 'Babish_knows_location_treasure', 'Babish_has_access_pirates', 'Charlotte_power', 'Charlotte_injured', 'Charlotte_state', 'Charlotte_at', 'Charlotte_treasure', 'Charlotte_altruism', 'Charlotte_ambition', 'Charlotte_knows_location_treasure', 'Charlotte_has_access_pirates', 'ship_distraction', 'pirateBase_distraction', 'treasurePlace_distraction']
		self.utilities = {
	'Anna':'satisfaction(Anna);\n',
	'Babish':'satisfaction(Babish);\n',
	'Charlotte':'satisfaction(Charlotte);\n',

		}	
		
		self.df_effects = pd.read_csv('..\\rules\\ship_effects_nn.csv')
		self.file = 'rl_planner_ship.txt'
		self.df_actions = pd.read_csv('..\\rules\\df_actions.csv')
		self.acting_character = self.characters[0]
		self.randomize_df()
		
		self.create_file(self.df)
		self.no_solution = 0
		self._nb_features = len(self.df.index) 
		print(self._nb_features)
		
		self.observation_space = spaces.Box(
			-1,
			3,
			shape = [self._nb_features]
		)

			

		self.target_feature = None
		self.target_value = None

		

		#possible actions
		self.action_space = spaces.Discrete(3)
		self._action_to_direction = {
			0: 'char0',
			1: 'char1',
			2: 'char2',
			
		}


		

		

	def change_number_to_cat(self,row):
		for index,value in row.items():
			if value != -1:
				
				if 'at' in index and 'state' not in index and 'location' not in index and 'relation' not in index and 'pirate' not in index:
					row[index] = self.number_to_place[str(value)]

		return row

	def change_cat_to_number(self,row):
		for index,value in row.items():
			if value != -1:
				
				if 'at' in index and 'state' not in index and 'location' not in index and 'relation' not in index and 'pirate' not in index:
					if str(value) in self.place_to_number.keys():
						row[index] = self.place_to_number[str(value)]

		return row


	def randomize_df(self):
		if self.scenario != None:
			num = self.scenario
		else:
			num  = random.randint(0, 2)
		print('scenario ',num)
		start_state =[]
		
		
		not_character = ['Goal','Real']
		

		lists = []
		columns = []

		columns.append("world_state")
		#lists.append([0,1,2])
		lists.append([2])

		columns.append("pirates_relation")
		#lists.append([0,1])
		lists.append([1])

		columns.append("pirates_power")
		#lists.append([0,1,2])
		lists.append([2])

		columns.append("pirates_treasure")
		#lists.append([0,1,2])
		lists.append([0])

		columns.append("pirates_knows_location_treasure")
		lists.append([1])

		for c in self.characters:
			columns.append(c+"_power")
			lists.append([0,1])

			columns.append(c+"_injured")
			lists.append([0])

			columns.append(c+"_state")
			lists.append([0,2])

			columns.append(c+"_at")
			lists.append(self.places)

			columns.append(c+"_treasure")
			if c== 'Babish':
				lists.append([0])
			elif c== 'Charlotte':
				lists.append([1])
			else:
				lists.append([0])

			columns.append(c+"_altruism")
			lists.append([1])

			columns.append(c+"_ambition")
			lists.append([0])

			columns.append(c+"_knows_location_treasure")
			
			lists.append([0])

			columns.append(c+"_has_access_pirates")
			lists.append([0])
		
			
				
		

		for p in self.places:
			columns.append(p+"_distraction")
			lists.append([0,1])
			
		for l in lists:
			start_state.append(random.choice(l))
				

		
		state = {
		'mind_level' : 0,
		'goal_utility' : 1}

		for i,know in enumerate(self.chars_know,0):
			state['Goal:'+str(know)] = -1
			state['Real:'+str(know)] = start_state[i]

			for char in self.characters:
				state[str(char)+'_believes:'+str(know)] = start_state[i]
		
		
		if num == 0 :
			state['Goal:Anna_treasure'] = 1

		elif num == 1:
			state['Goal:Babish_treasure'] = 1
			
		elif num == 2:
			state['Goal:Charlotte_treasure'] = 1
			
		
		
		
		
		self.df = pd.Series(state )
		
		
	def create_file(self,row):
		acting_character = self.acting_character
		with open(self.file, 'w') as f:
			f.write("""type place;
type pirate;
type world;

property power(character : character) : number;
property injured(character : character) : number;
property satisfaction(character : character) : number;
property state(character : character) : number; //0 - nic 1- odwrócenie uwagi 2 -ogłuszenie
property at(character : character) : place;
property treasure(character : character) : number;
property altruism(character : character) : number;
property ambition(character : character) : number;
property knows_location_treasure(character : character) : number;
property has_access_pirates(character : character) : number;

property world_state(world : world) : number; //dzień, noc, słaba pogoda 
property later(world : world) : boolean;
property pirates_knows_location_treasure(pirate : pirate) : number;
property pirates_relation(pirate : pirate) : number; //słaba, neutralna, dobra
property pirates_power(pirate : pirate) : number; 
property pirates_treasure(pirate : pirate) : number; 

property distraction(place:place) :number;

entity Anna : character;
entity Babish : character;
entity Charlotte : character;

entity world:world;
entity pirates:pirate;

entity  ship : place;
entity  pirateBase : place;
entity  treasurePlace : place;


		   

		   """
			)
			
			f.write("world_state(world) = "  + str(row['Real:'+"world_state"]) + " ;\n")
			f.write("pirates_relation(pirates) = "  + str(row['Real:'+"pirates_relation"]) + " ;\n")
			f.write("pirates_power(pirates) = "  + str(row['Real:'+"pirates_power"]) + " ;\n")
			f.write("pirates_treasure(pirates) = "  + str(row['Real:'+"pirates_treasure"]) + " ;\n")
			f.write("pirates_knows_location_treasure(pirates) = "  + str(row['Real:'+"pirates_knows_location_treasure"]) + " ;\n")
			
			for char in self.characters:
				
				f.write("power(" + char  +") = " + str(row['Real:'+char+"_power"]) + " ;\n")
				f.write("injured(" + char  +") = " + str(row['Real:'+char+"_injured"]) + " ;\n")
				f.write("state(" + char  +") = " + str(row['Real:'+char+"_state"]) + " ;\n")
				f.write("at(" + char  +") = " + str(row['Real:'+char+"_at"]) + " ;\n")
				f.write("treasure(" + char  +") = " + str(row['Real:'+char+"_treasure"]) + " ;\n")
				f.write("altruism(" + char  +") = " + str(row['Real:'+char+"_altruism"]) + " ;\n")
				f.write("ambition(" + char  +") = " + str(row['Real:'+char+"_ambition"]) + " ;\n")
				f.write("knows_location_treasure(" + char  +") = " + str(row['Real:'+char+"_knows_location_treasure"]) + " ;\n")
				f.write("has_access_pirates(" + char  +") = " + str(row['Real:'+char+"_has_access_pirates"]) + " ;\n")
			
				
			for p in self.places:
				f.write("distraction(" + p  +") = " + str(row['Real:'+p+"_distraction"]) + " ;\n")
				

			f.write("""
action travel(character : character,  to : place){{
	precondition:
		(
		(to==pirateBase & has_access_pirates(character)==1)
		|
		(to==treasurePlace & knows_location_treasure(character) ==1)
		|
		(to==ship)
		)
		& character == {0}
		;
	effect:
		later(world)&
		at(character) = to;
	consenting: character; 
	
}};

action distraction(character : character,  place : place){{
	precondition:
		at(character) == place & character == {0};
		
	effect:
		distraction(place) =1 & later(world);
	consenting: character; 
	
}};

action stun(character : character,character2 : character, place : place){{
	precondition:
		at(character) == place &
		at(character2) == place &
		distraction(place)==1 &
		character != character2 
		
		& character == {0};
		
	effect:
		if (altruism(character) > 0)
			satisfaction(character) = satisfaction(character) - 1 
		&state(character2) = 2
		& later(world);
	consenting: character; 
	
}};


action escape(character : character){{
	precondition:
		at(character) == ship &
		distraction(ship) == 1
		
		
		& character == {0};
		
	effect:
		if (ambition(character) == 0 | injured(character) == 1) 
			satisfaction(character) = satisfaction(character) + 1
		& later(world);
	consenting: character; 
	
}};

action escape_pirates_help(character : character){{
	precondition:
		at(character) == ship &
		(distraction(ship) == 1 & pirates_relation(pirates) == 1) |
		pirates_power(pirates) == 0 |
		world_state(world) > 0 
		
		& character == {0};
		
	effect:
		if (ambition(character) == 0 | injured(character) == 1) 
			satisfaction(character) = satisfaction(character) + 1
		& later(world);
	consenting: character; 
	
}};
action take_treasure_char(character : character,character2 : character, place : place){{
	precondition:
		at(character) == place &
		at(character2) == place &
		state(character2) == 2 &
		character != character2 &
		treasure(character2) > 0
		& character == {0};
		
	effect:
		if (altruism(character) > 0)
			satisfaction(character) = satisfaction(character) - 1 
		&
		if (ambition(character) > 0)
			satisfaction(character) = satisfaction(character) + 1 
		&
		treasure(character2) = treasure(character2) - 1 &
		treasure(character) = treasure(character) + 1
		& later(world);
	consenting: character; 
	
}};

action take_treasure_location_char(character : character,character2 : character, place : place){{
	precondition:
		at(character) == place &
		at(character2) == place &
		state(character2) == 2 &
		character != character2 &
		knows_location_treasure(character2)==1
		& character == {0};
		
	effect:
		if (altruism(character) > 0)
			satisfaction(character) = satisfaction(character) - 1 
		&
		
		knows_location_treasure(character)=1
		& later(world);
	consenting: character; 
	
}};

action get_access_pirates(character : character){{
	precondition:
		pirates_relation(pirates) == 1 |
		pirates_power(pirates) == 0 |
		world_state(world) > 0 
		& character == {0}
		;
	effect:
		later(world)&
		has_access_pirates(character) = 1;
	consenting: character; 
	
}};

action tell_treasure_location(character : character,character2 : character){{
	precondition:
		
		knows_location_treasure(character)==1
		& character == {0}
		;
	effect:
		later(world)&
		knows_location_treasure(character2)=1;
	consenting: character; 
	
}};


action get_treasure(character : character){{
	precondition:
		
		knows_location_treasure(character)==1
		&treasure(character) < 2
		& character == {0}
		;
	effect:
	
		if (ambition(character) > 0)
			satisfaction(character) = satisfaction(character) + 1 
		&
		later(world)&
		treasure(character) = treasure(character) + 1;
	consenting: character; 
	
}};

action give_pirates_treasure_location(character : character){{
	precondition:
		pirates_knows_location_treasure(pirates)==0 &
		knows_location_treasure(character)==1
		& character == {0}
		& at(character) == pirateBase
		;
	effect:
		later(world)&
		pirates_knows_location_treasure(pirates)==1 &
		pirates_relation(pirates) = pirates_relation(pirates) + 1;
	consenting: character; 
	
}};


action give_pirates_treasure(character : character){{
	precondition:
		treasure(character) > 0
		& character == {0}
		& at(character) == pirateBase
		;
	effect:
		if (ambition(character) > 0)
			satisfaction(character) = satisfaction(character) - 1 
		&
		later(world)&
		treasure(character) = treasure(character) - 1 &
		pirates_treasure(pirates) = pirates_treasure(pirates) + 1 &
		pirates_relation(pirates) = pirates_relation(pirates) + 1;
	consenting: character; 
	
}};

action take_pirates_treasure(character : character){{
	precondition:
		has_access_pirates(character) == 1
		& pirates_treasure(pirates) > 0
		& character == {0}
		;
	effect:
		if (ambition(character) > 0)
			satisfaction(character) = satisfaction(character) + 1 
		&
		later(world)&
		treasure(character) = treasure(character) + 1 &
		pirates_treasure(pirates) = pirates_treasure(pirates) - 1 &
		pirates_relation(pirates) = pirates_relation(pirates) - 1;
	consenting: character; 
	
}};



action attack_pirateBase(character : character){{
	precondition:
		power(character) > 0
		& character == {0}
		;
	effect:
		later(world)&
		pirates_power(pirates) = pirates_power(pirates) - 1
		 ; 
	consenting: character; 
	
}};

action train(character : character){{
	precondition:
		 character == {0}
		;
	effect:
		later(world)&
		power(character) = power(character) + 1
		 ; 
	consenting: character; 
	
}};


action take_treasure_pirates(character : character){{
	precondition:
		((pirates_power(pirates) == 0 ) | distraction(pirateBase) == 1 )
		& at(character) == pirateBase
		& character == {0}
		& pirates_treasure(pirates) > 0
		;
	effect:
	
		if (ambition(character) > 0)
			satisfaction(character) = satisfaction(character) + 1 
		&
		later(world)&
		pirates_treasure(pirates) = pirates_treasure(pirates) - 1
		&treasure(character) = treasure(character) + 1

		 ; 
	consenting: character; 
	
}};



			""".format(acting_character))


			df_act = self.df_actions[self.df_actions['char']!= 'char1']
			for index,row in df_act.iterrows():
				f.write(row.actions)

			
			
			f.write("utility(): \n ")
			f.write(self.utilities[acting_character])

			
		


			for char in self.characters:
				f.write("utility({}): \n ".format(char))
				f.write(self.utilities[char])
			
			
			f.close()


	def _get_obs(self):
		
		df2 = self.change_cat_to_number(self.df) 
		r = df2.to_numpy(dtype='float32')
		#print(len(r))
		return r
	
	def _get_info(self):
		return {0:'test'}

	def reset(self,seed = None,  options=None):
		
		print(self.scenario)
		super().reset()
		print('reset!')
		#randomize row
		self.mind_level=0
		self.randomize_df()
		#self.df = self.change_number_to_cat(self.df)
		self.create_file(self.df)
		self.no_solution = 0
		self.score = 0
		self.acting_character = self.characters[0]
		observation = self._get_obs() #turn row into observation
		info = self._get_info() #turn row into info
		
		
		return observation, info
	
	
	def load_action(self,file):

		
		p = Popen(['java', '-jar', 'E:\\Praca_magisterska\\narrative_system\\ship\\planners\\sabre.jar', '-p', file,'-el',"0","-h","h+",'-c','n',"-tl","3100"], stdout=PIPE, stderr=STDOUT)
		#p = Popen(['java', '-jar', 'lib\sabre.jar', '-p', file,'-el',"0",'-g',"","-tl","1000"], stdout=PIPE, stderr=STDOUT)

		lines=[]
		for line in p.stdout:
			lines.append(str(line, encoding='utf-8'))

		#print(lines)
		return lines[0].replace("\r\n","")

	def do_action(self,args):
		if len(args) > 0 and len(self.df_effects[self.df_effects.action == args[0] ]['effect_function'].values) > 0:
			functions = self.df_effects[self.df_effects.action == args[0] ]['effect_function'].values[0].split(';')
			for function in functions:
				parts = function.split(':')
				
				
				#choosing feature
				feature = parts[0]

				#choosing how feature is changed
				change = parts[1]

				for count,arg in enumerate(args,0):
					
						feature = feature.replace('arg'+str(count),arg)
					
						change = change.replace('arg'+str(count),arg)

				old_change = change
				change = change.split("_")[0]
				
				

				chars_observing = self.characters
				features_to_change = [char+'_believes' for char in chars_observing]
				features_to_change.append('Real')
				
				#Alice_believes:Alice_alive
				if  "=" in change[0] or '_at' in feature :
					
					for feat in features_to_change:
						
						for index in self.df.index:
							if (feat+':'+feature) in index and 'Goal' not in index:
								if '_at' in index:
									
									self.df[index] = self.place_to_number[old_change.split('_')[1]]
								else:
								
									self.df[index] = old_change[1:]
					#print(self.df[feature])
					
					
				elif "+" in change[0]:
					for feat in features_to_change:

						for index in self.df.index:
							if (feat+':'+feature) in index and 'Goal' not in index:
								self.df[index] = int(self.df[index]) + int(change[1])
								
								#self.df[feature] = int(self.df[feature].values[0]) + int(change[1])
				elif "-" in change[0]:
					for feat in features_to_change:

						for index in self.df.index:
							if (feat+':'+feature) in index and 'Goal' not in index:
								self.df[index] = int(self.df[index]) - int(change[1])
								#self.df[feature] = int(self.df[feature].values[0]) - int(change[1])
								
		
				#print(self.df.to)
				self.df = self.change_cat_to_number(self.df)
			

	def change_state(self,actions):
		
		
		
		
		if 'No solution' not in actions:
			actions = actions.split(') ')
			
			if len(actions) > 0:
				args = actions[0].replace("("," ").replace(")","").replace(",","")
				print(args)
				if ('key_action' not in args) :
					self.do_action(args.split(" "))
				
				return True
		
		return False

	def make_action(self,action):
		self.mind_level=0
		if 'mind_char' in action:
			char = action.split('_')[1]
			num = char.split('char')[1]
			self.acting_character = self.number_to_character[str(num)]
			print('mind level 1')
			self.mind_level=1
		elif 'ut_char' in action:
			sp = action.split('ut_char')[1]
			char = sp.split('_')[0]
			ut =  sp.split('_')[1]
			self.acting_character = self.number_to_character[str(char)]
			print(self.acting_character)
			if ut == "1":
				self.utilities = self.utilities1
			elif ut == "2":
				self.utilities = self.utilities2
			#self.additional_utility = action
		elif 'char' in action:
			char = action.split('char')[1]
			self.acting_character = self.number_to_character[str(char)]
			print(self.acting_character)
		
		
		else: print("nie ma takiej akcji!")
		
		
		self.create_file(self.change_number_to_cat(self.df))
		start = time.time()

		
		
		self.results = self.load_action(self.file)
		

		end = time.time()
		
		index = 0
		#self.df.loc[index,['results']] = self.results
		#self.df.loc[index,['time']] = (end-start)

		print(self.results)
		self.change_state(self.results)
		
	def get_reward(self):
		
		reward = 0
		
		sum = 0
		for know in self.chars_know:
			if str(self.df['Real:' + str(know)]) == str(self.df['Goal:' + str(know)]):
				sum +=1
				print('osiagniete')
		
		if sum  >= self.df['goal_utility']:
		
			reward = 1
			print('sukces!')
		elif 'No solution' in self.results or 'Time limit' in self.results:
			reward = -0.5
			self.no_solution = self.no_solution+1
		else: 
			self.no_solution = 0
			
		
		
		return reward

	def step(self, action):
		
		direction = self._action_to_direction[action]
		
		self.make_action(direction)
		
		terminated = False
		reward = self.get_reward()
		if reward > 0 :
			terminated = True
		if self.no_solution > 6:
			terminated = True
		self.score += reward
		observation = self._get_obs()
		info = self._get_info()

		
			

		return observation, reward, terminated, False, info

	
	
	

In [7]:
from gymnasium.envs.registration import register
# Example for the CartPole environment
register(
    # unique identifier for the env `name-version`
    id="ShipEnv",
    # path to the class for creating the env
    # Note: entry_point also accept a class as input (and not only a string)
    entry_point=ShipEnv3,
    # Max number of steps per episode, using a `TimeLimitWrapper`
    max_episode_steps=13,
)

e:\Praca_magisterska\narrative_system\.venv\lib\site-packages\gymnasium\envs\registration.py:694: UserWarning: WARN: Overriding environment ShipEnv already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [8]:
tmp_dir = "./rl_final_dqn_ship/"
os.makedirs(tmp_dir, exist_ok=True)

In [9]:
env =  gym.make("ShipEnv")
env = Monitor(env, tmp_dir)

scenario  0
177


In [11]:
import os

import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt

from stable_baselines3 import TD3
from stable_baselines3.common import results_plotter
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.results_plotter import load_results, ts2xy, plot_results
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.callbacks import BaseCallback


class SaveOnBestTrainingRewardCallback(BaseCallback):
    """
    Callback for saving a model (the check is done every ``check_freq`` steps)
    based on the training reward (in practice, we recommend using ``EvalCallback``).

    :param check_freq:
    :param log_dir: Path to the folder where the model will be saved.
      It must contains the file created by the ``Monitor`` wrapper.
    :param verbose: Verbosity level: 0 for no output, 1 for info messages, 2 for debug messages
    """
    def __init__(self, check_freq: int, log_dir: str, verbose: int = 1):
        super().__init__(verbose)
        self.check_freq = check_freq
        self.log_dir = log_dir
        self.save_path = os.path.join(log_dir, "best_model")
        self.best_mean_reward = -np.inf

    def _init_callback(self) -> None:
        # Create folder if needed
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self) -> bool:
        if self.n_calls % self.check_freq == 0:

          # Retrieve training reward
          x, y = ts2xy(load_results(self.log_dir), "timesteps")
          if len(x) > 0:
              # Mean training reward over the last 100 episodes
              mean_reward = np.mean(y[-100:])
              if self.verbose >= 1:
                print(f"Num timesteps: {self.num_timesteps}")
                print(f"Best mean reward: {self.best_mean_reward:.2f} - Last mean reward per episode: {mean_reward:.2f}")

              # New best model, you could save the agent here
              if mean_reward > self.best_mean_reward:
                  self.best_mean_reward = mean_reward
                  # Example for saving best model
                  if self.verbose >= 1:
                    print(f"Saving new best model to {self.save_path}")
                  self.model.save(self.save_path)

        return True


In [12]:
from stable_baselines3 import SAC
from stable_baselines3.common.callbacks import CheckpointCallback

# Save a checkpoint every 1000 steps
checkpoint_callback = CheckpointCallback(
  save_freq=200,
  save_path=tmp_dir,
  name_prefix="rl_model",
  save_replay_buffer=True,
  save_vecnormalize=True,
)

In [13]:
tensorboard_log = tmp_dir

In [14]:
from stable_baselines3.common.logger import configure

# set up logger
new_logger = configure(tmp_dir, ["stdout", "csv"])

Logging to ./rl_final_dqn_ship/


In [15]:
callback = SaveOnBestTrainingRewardCallback(check_freq=500, log_dir=tmp_dir)
callbacks = [callback,checkpoint_callback]
dqn_model = DQN(
    "MlpPolicy",
    env,
    verbose=1,
    train_freq=16,
    gradient_steps=8,
    gamma=0.99,
    exploration_fraction=0.4,
    exploration_final_eps=0.07,
    target_update_interval=600,
    learning_starts=1000,
    buffer_size=10000,
    batch_size=128,
    learning_rate=4e-3,
    policy_kwargs=dict(net_arch=[170, 170]),
    tensorboard_log=tmp_dir,
    
)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [16]:
dqn_model.learn(int(5000), log_interval=100,callback=callbacks)

1
reset!
scenario  1
Logging to ./rl_final_dqn_ship/DQN_37
Babish
escape_pirates_help(Babish)
escape_pirates_help Babish
Charlotte
escape_pirates_help(Charlotte)
escape_pirates_help Charlotte
Anna
travel(Anna, ship) escape_pirates_help(Anna)
travel Anna ship
Babish
escape_pirates_help(Babish)
escape_pirates_help Babish
Charlotte


KeyboardInterrupt: 

In [544]:
mean_reward, std_reward = evaluate_policy(dqn_model, dqn_model.get_env(), deterministic=True, n_eval_episodes=20)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

None
reset!
scenario  1
Babish
get_treasure(Babish)
get_treasure Babish
Babish
take_pirates_treasure(Babish)
take_pirates_treasure Babish
osiagniete
sukces!
None
reset!
scenario  1
Anna
get_treasure(Anna)
get_treasure Anna
Babish
get_access_pirates(Babish) take_pirates_treasure(Babish)
get_access_pirates Babish
Anna
get_treasure(Anna)
get_treasure Anna
Babish
take_pirates_treasure(Babish)
take_pirates_treasure Babish
osiagniete
sukces!
None
reset!
scenario  0
Anna
take_pirates_treasure(Anna)
take_pirates_treasure Anna
osiagniete
osiagniete
sukces!
None
reset!
scenario  1
Charlotte
take_treasure_char(Charlotte, Anna, pirateBase)
take_treasure_char Charlotte Anna pirateBase
osiagniete
sukces!
None
reset!
scenario  0
Babish
travel(Babish, treasurePlace) take_treasure_char(Babish, Anna, treasurePlace)
travel Babish treasurePlace
Babish
take_treasure_char(Babish, Anna, treasurePlace)
take_treasure_char Babish Anna treasurePlace
Babish
take_treasure_char(Babish, Anna, treasurePlace)
take_tre

In [545]:
dqn_model.save(tmp_dir+"/DQN_final_5.zip")